In [1]:
import findspark # type: ignore
import pandas as pd # type: ignore
import re
from pyspark.sql import SparkSession # type: ignore
from pyspark.sql import functions as F # type: ignore
from pyspark.sql.functions import col,udf # type: ignore
from pyspark.sql.types import IntegerType,StringType # type: ignore
from unidecode import unidecode # type: ignore

In [2]:
findspark.init()

df_senasa = pd.read_excel(r"Diccionario-Senasa_2024-09-28_CC.xlsx")
#df_veritrade=pd.read_excel("Veritrade_CMEDRANO@BIOMONT.COM.PE_PE_I_20240828163144_LIDA.xlsx",header=5)
#df_veritrade=pd.read_excel(r"Data_ExportacionesCompetidoresGeriax.xlsx",header=5)
df_veritrade=pd.read_excel(r"Data_ExportacionesCompetidoresGeriax.xlsx")

import os

print("SPARK_HOME:", os.environ.get('SPARK_HOME'))
print("HADOOP_HOME:", os.environ.get('HADOOP_HOME'))

In [3]:
#print(df_veritrade.dtypes)

PYSPARK ACEPTA SOLO FECHAS ENTRE 1970 y 2038

spark = SparkSession.builder \
    .appName("Pandas to PySpark DataFrame") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

#spark.conf.set("spark.network.timeout", "800s")
#spark.conf.set("spark.sql.shuffle.partitions", "200")

df_senasa_spark = spark.createDataFrame(df_senasa)
df_veritrade_spark = spark.createDataFrame(df_veritrade)

In [4]:
spark = SparkSession.builder \
    .appName("Basic Example") \
    .config("spark.driver.memory", "3g") \
    .config("spark.executor.memory", "3g") \
    .config("spark.executor.cores", "2") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "1g") \
    .config("spark.driver.maxResultSize","0") \
    .getOrCreate()

df_senasa_spark = spark.createDataFrame(df_senasa)
df_veritrade_spark = spark.createDataFrame(df_veritrade)

24/11/19 13:05:56 WARN Utils: Your hostname, hadoop resolves to a loopback address: 127.0.1.1; using 192.168.1.103 instead (on interface enp0s3)
24/11/19 13:05:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 13:05:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#!pip install --upgrade setuptools wheel

In [6]:
df_senasa_spark=df_senasa_spark.repartition(100)
df_senasa_spark=df_senasa_spark.coalesce(50)

df_veritrade_spark=df_veritrade_spark.repartition(100)
df_veritrade_spark=df_veritrade_spark.coalesce(50)

In [7]:
print("Driver Memory: ", spark.sparkContext.getConf().get("spark.driver.memory"))
print("Executor Memory: ", spark.sparkContext.getConf().get("spark.executor.memory"))
print("Executor Memory: ", spark.sparkContext.getConf().get("spark.executor.cores"))
print("Executor Memory: ", spark.sparkContext.getConf().get("spark.memory.offHeap.enabled"))
print("Executor Memory: ", spark.sparkContext.getConf().get("spark.memory.offHeap.size"))

Driver Memory:  3g
Executor Memory:  3g
Executor Memory:  2
Executor Memory:  true
Executor Memory:  1g


In [8]:
num_particiones_veritrade = df_senasa_spark.rdd.getNumPartitions()
print(f"Número de particiones en df_veritrade_spark: {num_particiones_veritrade}")

Número de particiones en df_veritrade_spark: 50


In [9]:
#df_senasa_spark.show(1)

#df_veritrade_spark.show(1)

Implementar lógica restante del veritrade

In [10]:
df_senasa_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- N_PROD_PAGINA: long (nullable = true)
 |-- TIPO_PRODUCTO: string (nullable = true)
 |-- NOMBRE_COMERCIAL: string (nullable = true)
 |-- REPETIDO: long (nullable = true)
 |-- EMPRESA_RESPONSABLE_REGISTRO: string (nullable = true)
 |-- N_REGISTRO: string (nullable = true)
 |-- FECHA_VENCIMIENTO: string (nullable = true)
 |-- ETIQUETA: long (nullable = true)
 |-- VAR1: string (nullable = true)
 |-- DETALLE_VAR1: string (nullable = true)
 |-- VAR2: string (nullable = true)
 |-- DETALLE_VAR2: string (nullable = true)
 |-- VAR3: string (nullable = true)
 |-- DETALLE_VAR3: string (nullable = true)
 |-- VAR4: string (nullable = true)
 |-- DETALLE_VAR4: string (nullable = true)
 |-- VAR5: string (nullable = true)
 |-- DETALLE_VAR5: string (nullable = true)
 |-- VAR6: string (nullable = true)
 |-- DETALLE_VAR6: string (nullable = true)
 |-- VAR7: string (nullable = true)
 |-- DETALLE_VAR7: string (nullable = true)
 |-- VAR8: string (nullable = true)
 |--

In [11]:
df_veritrade_spark.printSchema()

root
 |-- Partida Aduanera: long (nullable = true)
 |-- Descripcion de la Partida Aduanera: string (nullable = true)
 |-- Aduana: string (nullable = true)
 |-- DUA / DAM: string (nullable = true)
 |-- Fecha: timestamp (nullable = true)
 |-- Cod. Tributario: long (nullable = true)
 |-- Exportador: string (nullable = true)
 |-- Importador: string (nullable = true)
 |-- Kg Bruto: double (nullable = true)
 |-- Kg Neto: double (nullable = true)
 |-- Qty 1: double (nullable = true)
 |-- Und 1: string (nullable = true)
 |-- Qty 2: long (nullable = true)
 |-- Und 2: string (nullable = true)
 |-- U$ FOB Tot: double (nullable = true)
 |-- U$ FOB Und 1: double (nullable = true)
 |-- U$ FOB Und 2: double (nullable = true)
 |-- Pais de Destino: string (nullable = true)
 |-- Puerto de destino: string (nullable = true)
 |-- Último Puerto Embarque: double (nullable = true)
 |-- Via: string (nullable = true)
 |-- Agente Portuario: string (nullable = true)
 |-- Agente de Aduana: string (nullable = true)

In [12]:
caracteres_especiales = set()

def encontrar_caracteres_especiales(nombre):
    return {caracter for caracter in nombre if not re.match(r'[A-Za-z0-9 ]', caracter)}

#Obteniendo los caracteres especiales existentes en los nombres de los productos
caracteres_especiales = df_senasa_spark.select(col("NOMBRE_COMERCIAL")) \
    .rdd.flatMap(lambda row: encontrar_caracteres_especiales(row[0])) \
    .distinct() \
    .collect()

#print("Caracteres especiales encontrados:", caracteres_especiales)

In [13]:
def contar_palabras(texto):
    return len(texto.split()) if texto else 0

contar_palabras_udf = udf(contar_palabras, IntegerType())

df_senasa_spark = df_senasa_spark.withColumn("NUMERO_DE_PALABRAS", contar_palabras_udf(col("NOMBRE_COMERCIAL")))

In [14]:
#df_senasa_spark.select("NOMBRE_COMERCIAL", "NUMERO_DE_PALABRAS").show(3)

In [15]:
def estandarizar_texto(texto):
    texto=str(texto)
    resultado=texto.replace("MDCMTO.","")
    resultado=resultado.replace("MDCMTO","")
    resultado=resultado.replace("MDCTO.","")
    resultado=resultado.replace("MDCTO","")
    resultado=resultado.strip()
    resultado=resultado.upper()
    resultado=unidecode(resultado)
    caracteres_permitidos = ''.join(caracteres_especiales)
    resultado = re.sub(r'[^A-Za-z0-9 ' + re.escape(caracteres_permitidos) + ']', '', resultado)
    resultado = re.sub(r'\s*\d+(\.\d+)?\s*MM', '', resultado)
    resultado=resultado.replace("0","O")
    resultado=resultado.replace(","," ")
    resultado=resultado.replace("MATERIA PRIMA","")
    return resultado

estandarizar_texto_udf = udf(estandarizar_texto, StringType())

df_senasa_spark = df_senasa_spark.withColumn("DES_PROD", estandarizar_texto_udf(col("NOMBRE_COMERCIAL")))

In [16]:
#df_senasa_spark.select("ID","NOMBRE_COMERCIAL", "NUMERO_DE_PALABRAS","DES_PROD").show(3)

In [17]:
df_veritrade_spark=df_veritrade_spark.withColumn("DES_COM",estandarizar_texto_udf(col("Descripcion Comercial")))

In [18]:
num_particiones_veritrade = df_veritrade_spark.rdd.getNumPartitions()
print(f"Número de particiones en df_veritrade_spark: {num_particiones_veritrade}")

Número de particiones en df_veritrade_spark: 50


In [19]:
#df_veritrade_spark=df_veritrade_spark.coalesce(4)
#df_veritrade_spark.cache()

In [20]:
#df_veritrade_spark.select("Descripcion Comercial", "DES_COM").show(3)

In [21]:
df_senasa_filtrado = df_senasa_spark.filter(col("NUMERO_DE_PALABRAS")>=2)

dic_senasa = {row['DES_PROD']: row['ID'] for row in df_senasa_filtrado.select('DES_PROD', 'ID').collect()}

def detectar_producto(descripcion, producto):
    palabras_producto = producto.split()
    palabras_descripcion=descripcion.split()
    set_artificio=set({})
    for prod in palabras_producto:
        for des in palabras_descripcion:
            if des==prod:
                set_artificio.add(des)
    
    if set(palabras_producto)==set_artificio:
        return True
    else:
        return False

def obtener_foreign_key(texto):
    for nombre_producto in dic_senasa.keys():
        if detectar_producto(texto, nombre_producto):
            id_senasa = dic_senasa.get(nombre_producto)
            return id_senasa
    return None

obtener_foreign_key_udf = udf(obtener_foreign_key, IntegerType())

df_veritrade_spark = df_veritrade_spark.withColumn("FK_SENASA_ID", obtener_foreign_key_udf(col("DES_COM")))

In [22]:
#df_veritrade_spark.select("Descripcion Comercial", "DES_COM","FK_SENASA_ID").show(3)

#df_veritrade_filtrado = df_veritrade_spark.filter(col("FK_SENASA_ID").isNotNull())

#df_veritrade_filtrado.select("Descripcion Comercial", "DES_COM","FK_SENASA_ID").show(3)

In [23]:
df_senasa_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- N_PROD_PAGINA: long (nullable = true)
 |-- TIPO_PRODUCTO: string (nullable = true)
 |-- NOMBRE_COMERCIAL: string (nullable = true)
 |-- REPETIDO: long (nullable = true)
 |-- EMPRESA_RESPONSABLE_REGISTRO: string (nullable = true)
 |-- N_REGISTRO: string (nullable = true)
 |-- FECHA_VENCIMIENTO: string (nullable = true)
 |-- ETIQUETA: long (nullable = true)
 |-- VAR1: string (nullable = true)
 |-- DETALLE_VAR1: string (nullable = true)
 |-- VAR2: string (nullable = true)
 |-- DETALLE_VAR2: string (nullable = true)
 |-- VAR3: string (nullable = true)
 |-- DETALLE_VAR3: string (nullable = true)
 |-- VAR4: string (nullable = true)
 |-- DETALLE_VAR4: string (nullable = true)
 |-- VAR5: string (nullable = true)
 |-- DETALLE_VAR5: string (nullable = true)
 |-- VAR6: string (nullable = true)
 |-- DETALLE_VAR6: string (nullable = true)
 |-- VAR7: string (nullable = true)
 |-- DETALLE_VAR7: string (nullable = true)
 |-- VAR8: string (nullable = true)
 |--

df_senasa_spark.createOrReplaceTempView("senasa")
senasa = spark.sql("SELECT * FROM senasa")

df_veritrade_spark.createOrReplaceTempView("veritrade")
veritrade = spark.sql("SELECT * FROM veritrade")

query="""
SELECT ver.*,sen.DES_PROD,sen.TIPO_PRODUCTO,sen.DETALLE_VAR1 AS COMPOSICION,sen.DETALLE_VAR2 AS CLASIFICACION,sen.DETALLE_VAR3 AS FORMA
FROM veritrade ver
LEFT JOIN senasa sen
ON ver.FK_SENASA_ID=sen.ID
"""

df_veritrade_spark_ite1 = spark.sql(query)
df_veritrade_spark_ite1=df_veritrade_spark_ite1.coalesce(60)
df_veritrade_spark_ite1.cache()


In [24]:
df_veritrade_spark_ite1=df_veritrade_spark

In [25]:
from unidecode import unidecode # type: ignore

def estandarizar_texto2(texto):
    texto=str(texto)
    resultado=texto.replace("MDCMTO.","")
    resultado=resultado.replace("MDCMTO","")
    resultado=resultado.replace("MDCTO.","")
    resultado=resultado.replace("MDCTO","")
    resultado=resultado.strip()
    resultado=resultado.upper()
    resultado=unidecode(resultado)
    caracteres_permitidos = ''.join(" ")
    resultado = re.sub(r'[^A-Za-z0-9 ' + re.escape(caracteres_permitidos) + ']', ' ', resultado) #DEJA LETRAS Y NUMEROS Y ESPACIOS Y CAMBIA LOS CARACTERES NO PERMITIDOS POR ESPACIOS
    resultado = re.sub(r'\s*\d+(\.\d+)?\s*MM', '', resultado) #ELIMINAMOS NUMERO Y UNIDAD DE MEDIDA PUEDE CAUSAR CONFUNSION CON OTRO NOMBRE DE PRODUCTO MM 100
    resultado=resultado.replace("0","O")
    resultado=resultado.replace("MATERIA PRIMA","") #KEYWORD QUE PUEDEN CONFUNDIR CONSIDERAR QUE ESTA KEYWORD SOLO SE ELIMINE EN VERITRADE
    resultado = re.sub(r'\s+', ' ', resultado)  # Reemplazar múltiples espacios por uno
    return resultado

estandarizar_texto2_udf = udf(estandarizar_texto2, StringType())

df_senasa_spark = df_senasa_spark.withColumn("DES_PROD2", estandarizar_texto2_udf(col("NOMBRE_COMERCIAL")))

In [26]:
df_senasa_filtrado = df_senasa_spark.filter(col("NUMERO_DE_PALABRAS")>=2)

dic_senasa = {row['DES_PROD2']: row['ID'] for row in df_senasa_filtrado.select('DES_PROD2', 'ID').collect()}

df_veritrade_spark_ite1 = df_veritrade_spark_ite1.withColumn(
    "FK_SENASA_ID",
    F.when(col("FK_SENASA_ID").isNull(), obtener_foreign_key_udf(col("DES_COM"))).otherwise(col("FK_SENASA_ID"))
)


df_senasa_spark.createOrReplaceTempView("senasa")
senasa = spark.sql("SELECT * FROM senasa")

df_veritrade_spark_ite1.createOrReplaceTempView("veritrade")
veritrade = spark.sql("SELECT * FROM veritrade")

query="""
SELECT ver.*,sen.DES_PROD,sen.TIPO_PRODUCTO,sen.DETALLE_VAR1 AS COMPOSICION,sen.DETALLE_VAR2 AS CLASIFICACION,sen.DETALLE_VAR3 AS FORMA
FROM veritrade ver
LEFT JOIN senasa sen
ON ver.FK_SENASA_ID=sen.ID
"""

df_veritrade_spark_ite2 = spark.sql(query)
df_veritrade_spark_ite2=df_veritrade_spark_ite2.coalesce(60)

df_veritrade_spark_ite2.cache()

In [27]:
df_veritrade_spark_ite2=df_veritrade_spark_ite1

In [28]:
def estandarizar_texto3(texto):
    texto=str(texto)
    resultado=texto.replace("MDCMTO.","")
    resultado=resultado.replace("MDCMTO","")
    resultado=resultado.replace("MDCTO.","")
    resultado=resultado.replace("MDCTO","")
    resultado=resultado.strip()
    resultado=resultado.upper()
    resultado=unidecode(resultado)
    resultado = re.sub(r'[^A-Za-z0-9]', ' ', resultado)
    resultado = re.sub(r'\s*\d+(\.\d+)?\s*MM', '', resultado)
    resultado=resultado.replace("0","O")
    resultado = re.sub(r'\s+', ' ', resultado)
    resultado=resultado.replace("MATERIA PRIMA","") #KEYWORD QUE PUEDEN CONFUNDIR CONSIDERAR QUE ESTA KEYWORD SOLO SE ELIMINE EN VERITRADE
    return resultado

estandarizar_texto3_udf = udf(estandarizar_texto3, StringType())

df_senasa_spark = df_senasa_spark.withColumn("DES_PROD3", estandarizar_texto3_udf(col("NOMBRE_COMERCIAL")))

In [29]:
df_senasa_filtrado = df_senasa_spark.filter(col("NUMERO_DE_PALABRAS")==1)

dic_senasa = {row['DES_PROD3']: row['ID'] for row in df_senasa_filtrado.select('DES_PROD3', 'ID').collect()}

def detectar_producto2(descripcion, producto):
    descripcion=str(descripcion)
    producto=str(producto)

    if descripcion.startswith(producto+" "):
        return True
    else:
        return False

def obtener_foreign_key2(texto):
    for nombre_producto in dic_senasa.keys():
        if detectar_producto2(texto, nombre_producto):
            id_senasa = dic_senasa.get(nombre_producto)
            return id_senasa
    return None

obtener_foreign_key2_udf = udf(obtener_foreign_key2, IntegerType())


df_veritrade_spark_ite2 = df_veritrade_spark_ite2.withColumn(
    "FK_SENASA_ID",
    F.when(col("FK_SENASA_ID").isNull(), obtener_foreign_key2_udf(col("DES_COM"))).otherwise(col("FK_SENASA_ID"))
)

df_senasa_spark.createOrReplaceTempView("senasa")
senasa = spark.sql("SELECT * FROM senasa")

df_veritrade_spark_ite2.createOrReplaceTempView("veritrade")
veritrade = spark.sql("SELECT * FROM veritrade")

query="""
SELECT ver.*,sen.DES_PROD,sen.TIPO_PRODUCTO,sen.DETALLE_VAR1 AS COMPOSICION,sen.DETALLE_VAR2 AS CLASIFICACION,sen.DETALLE_VAR3 AS FORMA
FROM veritrade ver
LEFT JOIN senasa sen
ON ver.FK_SENASA_ID=sen.ID
"""

df_veritrade_spark_ite3 = spark.sql(query)
df_veritrade_spark_ite3=df_veritrade_spark_ite3.coalesce(60)

df_veritrade_spark_ite3.cache()

In [30]:
df_veritrade_spark_ite3=df_veritrade_spark_ite2

In [31]:
def estandarizar_texto4(texto):
    texto=str(texto)
    resultado=texto.replace("MDCMTO.","")
    resultado=resultado.replace("MDCMTO","")
    resultado=resultado.replace("MDCTO.","")
    resultado=resultado.replace("MDCTO","")
    resultado=resultado.strip()
    resultado=resultado.upper()
    resultado=unidecode(resultado)
    caracteres_permitidos = ''.join(caracteres_especiales)
    resultado = re.sub(r'[^A-Za-z0-9 ' + re.escape(caracteres_permitidos) + ']', '', resultado)
    resultado = re.sub(r'\s*\d+(\.\d+)?\s*MM', '', resultado)
    resultado=resultado.replace("0","O")
    resultado=resultado.replace(",","")
    resultado=resultado.replace(" ","")
    resultado=resultado.replace("MATERIA PRIMA","") #KEYWORD QUE PUEDEN CONFUNDIR CONSIDERAR QUE ESTA KEYWORD SOLO SE ELIMINE EN VERITRADE
    return resultado

estandarizar_texto4_udf = udf(estandarizar_texto4, StringType())

df_senasa_spark = df_senasa_spark.withColumn("DES_PROD4", estandarizar_texto4_udf(col("NOMBRE_COMERCIAL")))

dic_senasa = {row['DES_PROD4']: row['ID'] for row in df_senasa_spark.select('DES_PROD4', 'ID').collect()}

In [32]:
def detectar_producto3(descripcion, producto):
    descripcion=str(descripcion)
    producto=str(producto)

    if producto in descripcion:
        return True
    else:
        return False

def obtener_foreign_key3(texto):
    for nombre_producto in dic_senasa.keys():
        if detectar_producto3(texto, nombre_producto):
            id_senasa = dic_senasa.get(nombre_producto)
            return id_senasa
    return None

obtener_foreign_key3_udf = udf(obtener_foreign_key3, IntegerType())

df_veritrade_spark_ite3 = df_veritrade_spark_ite3.withColumn(
    "FK_SENASA_ID",
    F.when(col("FK_SENASA_ID").isNull(), obtener_foreign_key3_udf(col("DES_COM"))).otherwise(col("FK_SENASA_ID"))
)

In [33]:
df_veritrade_spark_ite4=df_veritrade_spark_ite3

In [34]:
def estandarizar_texto5(texto):
    texto=str(texto)
    resultado=texto.replace("MDCMTO.","")
    resultado=resultado.replace("MDCMTO","")
    resultado=resultado.replace("MDCTO.","")
    resultado=resultado.replace("MDCTO","")
    resultado=resultado.strip()
    resultado=resultado.upper()
    resultado=unidecode(resultado)
    resultado = re.sub(r'[^A-Za-z0-9]', '', resultado)
    resultado = re.sub(r'\s*\d+(\.\d+)?\s*MM', '', resultado)
    resultado=resultado.replace("0","O")
    resultado=resultado.replace(",","")
    resultado=resultado.replace(" ","")
    resultado=resultado.replace("MATERIA PRIMA","") #KEYWORD QUE PUEDEN CONFUNDIR CONSIDERAR QUE ESTA KEYWORD SOLO SE ELIMINE EN VERITRADE
    return resultado

estandarizar_texto5_udf = udf(estandarizar_texto5, StringType())

df_senasa_spark = df_senasa_spark.withColumn("DES_PROD5", estandarizar_texto4_udf(col("NOMBRE_COMERCIAL")))

dic_senasa = {row['DES_PROD5']: row['ID'] for row in df_senasa_spark.select('DES_PROD5', 'ID').collect()}

df_veritrade_spark_ite4 = df_veritrade_spark_ite4.withColumn(
    "FK_SENASA_ID",
    F.when(col("FK_SENASA_ID").isNull(), obtener_foreign_key3_udf(col("DES_COM"))).otherwise(col("FK_SENASA_ID"))
)

In [35]:
#df_veritrade_spark_ite4.write.csv("output.csv")

In [36]:
df_veritrade_spark_ite4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Partida Aduanera#50L, Descripcion de la Partida Aduanera#51, Aduana#52, DUA / DAM#53, Fecha#54, Cod. Tributario#55L, Exportador#56, Importador#57, Kg Bruto#58, Kg Neto#59, Qty 1#60, Und 1#61, Qty 2#62L, Und 2#63, U$ FOB Tot#64, U$ FOB Und 1#65, U$ FOB Und 2#66, Pais de Destino#67, Puerto de destino#68, Último Puerto Embarque#69, Via#70, Agente Portuario#71, Agente de Aduana#72, Descripcion Comercial#73, ... 16 more fields]
   +- BatchEvalPython [obtener_foreign_key3(DES_COM#185)#538], [pythonUDF0#584]
      +- Project [Partida Aduanera#50L, Descripcion de la Partida Aduanera#51, Aduana#52, DUA / DAM#53, Fecha#54, Cod. Tributario#55L, Exportador#56, Importador#57, Kg Bruto#58, Kg Neto#59, Qty 1#60, Und 1#61, Qty 2#62L, Und 2#63, U$ FOB Tot#64, U$ FOB Und 1#65, U$ FOB Und 2#66, Pais de Destino#67, Puerto de destino#68, Último Puerto Embarque#69, Via#70, Agente Portuario#71, Agente de Aduana#72, Descripcion Comercial#73, 

24/11/19 13:06:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [37]:
num_particiones_veritrade = df_veritrade_spark_ite4.rdd.getNumPartitions()
print(f"Número de particiones en df_veritrade_spark: {num_particiones_veritrade}")

Número de particiones en df_veritrade_spark: 50


In [38]:
from pyspark.sql.functions import broadcast
df_veritrade_spark_ite5=df_veritrade_spark_ite4.join(broadcast(df_senasa_spark),df_veritrade_spark_ite4.FK_SENASA_ID==df_senasa_spark.ID,"left")

In [39]:
df_veritrade_spark_ite5.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [cast(FK_SENASA_ID#539 as bigint)], [ID#0L], LeftOuter, BuildRight, false
   :- Project [Partida Aduanera#50L, Descripcion de la Partida Aduanera#51, Aduana#52, DUA / DAM#53, Fecha#54, Cod. Tributario#55L, Exportador#56, Importador#57, Kg Bruto#58, Kg Neto#59, Qty 1#60, Und 1#61, Qty 2#62L, Und 2#63, U$ FOB Tot#64, U$ FOB Und 1#65, U$ FOB Und 2#66, Pais de Destino#67, Puerto de destino#68, Último Puerto Embarque#69, Via#70, Agente Portuario#71, Agente de Aduana#72, Descripcion Comercial#73, ... 16 more fields]
   :  +- BatchEvalPython [obtener_foreign_key3(DES_COM#185)#538], [pythonUDF0#731]
   :     +- Project [Partida Aduanera#50L, Descripcion de la Partida Aduanera#51, Aduana#52, DUA / DAM#53, Fecha#54, Cod. Tributario#55L, Exportador#56, Importador#57, Kg Bruto#58, Kg Neto#59, Qty 1#60, Und 1#61, Qty 2#62L, Und 2#63, U$ FOB Tot#64, U$ FOB Und 1#65, U$ FOB Und 2#66, Pais de Destino#67, Puerto de destino#68,

In [40]:
num_particiones_veritrade = df_veritrade_spark_ite5.rdd.getNumPartitions()
print(f"Número de particiones en df_veritrade_spark: {num_particiones_veritrade}")

Número de particiones en df_veritrade_spark: 50


df_senasa_spark.createOrReplaceTempView("senasa")
senasa = spark.sql("SELECT * FROM senasa")

df_veritrade_spark_ite4.createOrReplaceTempView("veritrade")
veritrade = spark.sql("SELECT * FROM veritrade")

query="""
SELECT ver.*,sen.DES_PROD,sen.TIPO_PRODUCTO,sen.DETALLE_VAR1 AS COMPOSICION,sen.DETALLE_VAR2 AS CLASIFICACION,sen.DETALLE_VAR3 AS FORMA
FROM veritrade ver
LEFT JOIN senasa sen
ON ver.FK_SENASA_ID=sen.ID
"""

df_veritrade_spark_ite5 = spark.sql(query)
df_veritrade_spark_ite5=df_veritrade_spark_ite5.coalesce(4)

In [41]:
def verificacion_alimento_en_des(texto):
    texto = str(texto)
    if texto.startswith("ALIMENTO") or texto.startswith("ALMNTO.") or "ALIMENTO" in texto:
        return True
    return False

def indicar_alimento(descripcion_comercial):
    if verificacion_alimento_en_des(descripcion_comercial):
        return "Alimento"
    
indicar_alimento_udf = udf(indicar_alimento, StringType())

df_veritrade_spark_ite6 = df_veritrade_spark_ite5.withColumn(
    "TIPO_PRODUCTO",
    F.when(col("FK_SENASA_ID").isNull(), indicar_alimento_udf(col("DES_COM"))).otherwise(col("TIPO_PRODUCTO"))
)

In [42]:
#df_veritrade_spark_ite6.explain()

In [43]:
def verificacion_agricola_en_des(texto):
    texto = str(texto)
    if texto.startswith("AGRICOLA") or texto.startswith("AGRÍCOLA") or "AGRICULTURA" in texto:
        return True
    return False

def indicar_agricola(descripcion_comercial):
    if verificacion_alimento_en_des(descripcion_comercial):
        return "Agrícola"
    
indicar_agricola_udf = udf(indicar_agricola, StringType())

df_veritrade_spark_ite6 = df_veritrade_spark_ite6.withColumn(
    "TIPO_PRODUCTO",
    F.when(col("FK_SENASA_ID").isNull()| col("TIPO_PRODUCTO").isNull(), indicar_agricola_udf(col("DES_COM"))).otherwise(col("TIPO_PRODUCTO"))
)

In [44]:
#num_particiones_veritrade = df_veritrade_spark_ite6.rdd.getNumPartitions()
#print(f"Número de particiones en df_veritrade_spark: {num_particiones_veritrade}")

In [45]:
import re
import ast

#df_veritrade=pd.read_excel("VERITRADE_pruebas_01102024.xlsx")
#df_senasa=pd.read_excel("Diccionario-Senasa_2024-09-28_CC.xlsx",sheet_name="2024-09-28")

# Crear un diccionario de tipos de presentación
presentaciones = {
    'AMPOLLA':['AMPOLLA '],
    'BALDE':['BALDE '],
    'BANDEJA':['BANDEJA '],
    'BARRIL':['BARRIL '],
    'BIDON': ['BIDON ', 'BIDONES ', 'BIDON(ES) '],
    'BIG BAG': ['BIG BAG '],
    'BLISTER':['BLISTER '],
    'BLOQUE': ['BLOQUE ', 'BLOQUES ', 'BLOQUE(S) '],
    'BOLSA':['BOLSA ','BOLSAS '],
    'BOTELLA': ['BOTELLA ', 'BOTELLAS ', 'BOTELLA(S) ', 'BOTTLE ', 'BOTTLES ', 'BOTTLE(S) ', ' BOT '],
    'CAJA': ['CAJA ', 'CAJAS ', 'CAJA(S) '],
    'CILINDRO': ['CILINDRO ', 'CILINDROS ', 'CILINDRO(S) '],
    'COJIN': ['COJIN '],
    'COLLAR': ['COLLAR '],
    'CONTENEDOR': ['CONTENEDOR '],
    'DOY': ['DOY PACK '],
    'ENVASE': ['ENVASE ', 'ENVASES ', 'ENVASE(S) '],
    'ESTUCHE': ['ESTUCHE '],
    'FLEXITANK': ['FLEXITANK '],
    'FRASCO':['FRASCO ', 'FRASCOS ', 'FRASCO(S) ', "FCO. "],
    'GALON': ['GALON ', 'GALONES ', 'GALON(ES) ', 'GALONERA ', 'GALONERA(S) '],
    'GARRAFA': ['GARRAFA '],
    'GRANEL': ['GRANEL '],
    'ISOTANQUE': ['INSOTANQUE ','ISOTANQUES '],
    'JERINGA': ['JERINGA ', 'JERINGAS ', 'JERINGA(S) '],
    'LATA': ['LATA ','LATAS '],
    'PIPETA': ['SPOT ON ', 'RIVOLTA ', 'PIPETA ', 'PIPETAS ', 'PIPETA(S) '],
    'POMO': ['POMO ', 'POMOS ', 'POMO(S) '],
    'POTE': ['POTE ', 'POTES ', 'POTE(S) '],
    'POUCH': ['POUCH '],
    'SACHET': ['SACHET '],
    'SACO':['SACO ', 'SACOS ', 'SACO(S) '],
    'SOBRE': ['SOBRE ', 'SOBRES ', 'SOBRE(S) '],
    'TAMBOR': ['TAMBOR ', 'TAMBORES ', 'TAMBOR(ES) ', 'DRUM ', 'DRUMS ', 'DRUM(S) '],
    'TANQUE': ['TANQUE '],
    'TARRO': ['TARRO '],
    'TUBO': ['TUBO ', 'TUBOS ', 'TUBO(S) '],
    'VIAL': ['VIAL '],
}

unidades_dict = {
    'KG': ['KG','KGS', 'KILOGRAMOS', 'KILOS'],
    'GR': ['GR','GRAMOS','G','GRMS','GRS'],
    'LT': ['LT', 'LTR','LTS','LTRS','LTROS', 'LITROS', 'LITRO','L'],
    'ML': ['ML'],
    #'UND':['UND'], #YA SABES QUE ES UNA UNIDAD PERO COMO SE SI ES TABLETA O COMPRIMIDO
    #'TAB':['TAB','TABS','TABLETA','TABLETAS','PASTILLAS','COMP']
}

sinonimos_a_clave = {}
for clave, sinonimos in unidades_dict.items():
    for sinonimo in sinonimos:
        sinonimos_a_clave[sinonimo] = clave

#pattern = r'(?<!\S)(\d+(?:[.,]\d+)?)\s*(' + '|'.join(re.escape(unit.strip()) for units in unidades_dict.values() for unit in units) + r')\b'
pattern = r'(?<!\S)(\d+(?:[.,]\d+)?)\s*(' + '|'.join(re.escape(unit) for unit in sinonimos_a_clave.keys()) + r')\b'
pattern_solo_valor=r'(?<!\S)(\d+(?:[.,]\d+)?)\s*\b'

senasa_data = df_senasa_spark.select("ID", "DETALLE_VAR4", "DETALLE_VAR3").collect()
senasa_dict = {row.ID: (row.DETALLE_VAR4, row.DETALLE_VAR3) for row in senasa_data}

def primeras_presentaciones_udf(id_senasa):
    if id_senasa is None or id_senasa not in senasa_dict:
        return
    
    presentacion, forma = senasa_dict[id_senasa]

    if isinstance(presentacion, str):
        try:
            my_pre_list = ast.literal_eval(presentacion)
        except (ValueError, SyntaxError):
            return
        
        if len(my_pre_list) == 1:
            presentacion = my_pre_list[0].upper()
            coincidencias = re.findall(pattern, presentacion)
            valor = re.findall(pattern_solo_valor, presentacion)

            if coincidencias:
                resultados = [f"{cantidad.replace(',', '.')} {sinonimos_a_clave[unidad.strip()]}" for cantidad, unidad in coincidencias]
                for k in presentaciones.keys():
                    if k in my_pre_list[0]:
                        return f"{k} X {resultados[0]}"
                    if "Und" in my_pre_list[0] and (forma in ["COMPRIMIDOS", "TABLETAS"]):
                        return f"{k} X {valor[0]} COMP"
            return
    return

# Registrar la UDF
primeras_presentaciones_udf = udf(primeras_presentaciones_udf, StringType())

# Aplicar la UDF al DataFrame de veritrade
df_veritrade_spark_ite7 = df_veritrade_spark_ite6.withColumn("PRESENTACION", primeras_presentaciones_udf(col("FK_SENASA_ID")))

In [46]:
from pyspark.sql.types import ArrayType

unidades_dict2 = {
    'KG': ['KG','KGS', 'KILOGRAMOS', 'KILOS'],
    'GR': ['GR','GRAMOS','G','GRMS','GRS'],
    'LT': ['LT', 'LTR','LTS','LTRS','LTROS', 'LITROS', 'LITRO','L'],
    'ML': ['ML'],
    'UND':["UND","UNIDADES","UNID."]
}

unidad_map = {unit: std for std, units in unidades_dict2.items() for unit in units}

def encontrar_unidad_medida(texto):
    texto = texto.upper()
    texto = texto.replace("X", " ")
    texto = texto.replace("/", " ")
    pattern = r'(?<!\S)(\d+(?:[.,]\d+)?)\s*(' + '|'.join(re.escape(unit.strip()) for units in unidades_dict2.values() for unit in units) + r')\b'
    coincidencias = re.findall(pattern, texto)
    resultados = [f"{cantidad.replace(',', '.')} {unidad_map[unidad.strip()]}" for cantidad, unidad in coincidencias if unidad.strip() in unidad_map]
    return resultados


encontrar_unidad_udf = udf(encontrar_unidad_medida, ArrayType(StringType()))

# Aplicar la UDF y crear una nueva columna
df_veritrade_spark_ite8 = df_veritrade_spark_ite7.withColumn("UNIDAD", encontrar_unidad_udf(col("Descripcion Comercial")))


In [47]:
df_veritrade_spark_ite9 = df_veritrade_spark_ite8.withColumn(
    "PRESENTACION",
    F.when(col("PRESENTACION").isNull(), indicar_agricola_udf(col("DES_COM"))).otherwise(col("PRESENTACION"))
)

presentaciones_vacias = df_veritrade[df_veritrade['PRESENTACION'] == '']['FK_SENASA_ID'].index.tolist()

sinonimos_a_clave_presentacion = {}
for clave, sinonimos in presentaciones.items():
    for sinonimo in sinonimos:
        sinonimos_a_clave_presentacion[sinonimo.strip()] = clave.strip()

sinonimos_a_clave_extra = {}
for clave, sinonimos in unidades_dict2.items():
    for sinonimo in sinonimos:
        sinonimos_a_clave_extra[sinonimo] = clave

pattern = r'(?<!\S)(\d+(?:[.,]\d+)?)\s*(' + '|'.join(re.escape(unit) for unit in sinonimos_a_clave_extra.keys()) + r')\b'
pattern_solo_valor = r'(?<!\S)(\d+(?:[.,]\d+)?)(?=\s*[a-zA-Z])'

def obtener_presentacion(cadena):
    cadena = cadena.strip()  # Eliminar espacios innecesarios
    return sinonimos_a_clave_presentacion.get(cadena, cadena)  # Devuelve la clave o la misma cadena si no se encuentra

def encontrar_presentaciones(texto, presentaciones):
    texto = texto.upper()
    encontrados = []

    for clave, sinonimos in presentaciones.items():
        for sinonimo in sinonimos:
            if sinonimo.strip() in texto:
                encontrados.append(clave)
                break

    return list(set(encontrados))


for idx in presentaciones_vacias:
    id_senasa = df_veritrade.loc[idx, "FK_SENASA_ID"]
    forma = df_senasa.loc[df_senasa["ID"] == id_senasa, "DETALLE_VAR4"].values

    valor_unidad_medida=[]

    if forma.size > 0:  # Asegúrate de que hay al menos un elemento
        forma_str = forma[0]  # Obtén el primer elemento
        if isinstance(forma_str, str):
            try:
                forma_list = ast.literal_eval(forma_str)
                my_set=set({})
                my_set_unidad=set({})
                for i in forma_list:
                    valor=i.split(" ")
                    if len(valor)>=3:
                        my_set.add(valor[0])
                        my_set_unidad.add(valor[-1])
                if len(my_set)==1:
                    my_pres=list(my_set)
                    my_pres=my_pres[0]  
                    forma_str=forma_str.upper()
                    coincidencias_valor_unidad_medida = re.findall(pattern, forma_str)
                    resultados = [f"{cantidad.replace(',', '.')} {sinonimos_a_clave_extra[unidad.strip()]}" for cantidad, unidad in coincidencias_valor_unidad_medida]    #TIPO LISTA
                    valor_unidad_medida_veritrade = df_veritrade.loc[idx, "UNIDAD"]
                    encontrado = False
                    if len(valor_unidad_medida_veritrade)==0:
                        pass
                    else:
                        #SOLO FUNCIONA PARA LOS CASOS QUE NO SEAN TAB NI COMP
                        for result in resultados:
                            for result2 in valor_unidad_medida_veritrade:
                                if result==result2:
                                    my_pres=obtener_presentacion(my_pres)
                                    cad_pres=my_pres+" X "+result2
                                    df_veritrade.loc[idx, "PRESENTACION"]=cad_pres
                                    encontrado=True
                                    break
                                else:
                                    pass
                            if encontrado:
                                break
                    
                    if (df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="TABLETAS" or df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="TABLETAS INTRAUTERINAS") and encontrado==False:
                        valor=re.findall(pattern_solo_valor, df_veritrade.loc[idx,"Descripcion Comercial"])
                        valor = [numero.replace(',', '.') for numero in valor]
                        for i in range(len(valor)):
                            valor[i]=valor[i]+" UND"
                        encontrado_tableta=False
                        for veri in valor:
                            for sena in resultados:
                                if veri==sena:
                                    my_pres=obtener_presentacion(my_pres)
                                    cad_pres=my_pres+" X "+sena
                                    cad_pres=cad_pres.replace("UND","TAB")
                                    df_veritrade.loc[idx, "PRESENTACION"]=cad_pres
                                    encontrado_tableta=True
                                    break
                            if encontrado_tableta:  # Si se encontró una coincidencia, salir del bucle externo
                                break
                        
                    if (df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="COMPRIMIDOS" or df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="COMPRIMIDOS MASTICABLES") and encontrado==False:
                        valor=re.findall(pattern_solo_valor, df_veritrade.loc[idx,"Descripcion Comercial"])
                        valor = [numero.replace(',', '.') for numero in valor]
                        for i in range(len(valor)):
                            valor[i]=valor[i]+" UND"
                        
                        encontrado_comprimido=False
                        for veri in valor:
                            for sena in resultados:
                                if veri==sena:
                                    my_pres=obtener_presentacion(my_pres)
                                    cad_pres=my_pres+" X "+sena
                                    cad_pres=cad_pres.replace("UND","COMP")
                                    df_veritrade.loc[idx, "PRESENTACION"]=cad_pres
                                    encontrado_comprimido=True
                                    #print(idx)
                                    break
                            
                            if encontrado_comprimido:  # Si se encontró una coincidencia, salir del bucle externo
                                break  
                else:
                    forma_str=forma_str.upper()
                    #print("tiene mas de una presentacion")
                    resultado_veritrade=encontrar_presentaciones(df_veritrade.loc[idx,"Descripcion Comercial"], presentaciones) #SI
                    resultado_senasa=encontrar_presentaciones(forma_str, presentaciones)                                        #SI
                    if len(resultado_veritrade)==0:
                        pass
                    else:
                        salir = False

                        for veri in resultado_veritrade:
                            if salir:  # Verifica si se debe salir
                                break
                            for sena in resultado_senasa:
                                if veri == sena:
                                    valor = re.findall(pattern_solo_valor, df_veritrade.loc[idx, "Descripcion Comercial"])
                                    valor = [numero.replace(',', '.') for numero in valor]
                                    for li in forma_list:
                                        for val in valor:
                                            if (li.startswith(sena)) and (val in li):
                                                partes = li.split(" ")
                                                parte_um = partes[-1].upper()

                                                if parte_um=="UND":
                                                    if (df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="TABLETAS" or df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="TABLETAS INTRAUTERINAS"):
                                                        cadena = f"{sena} X {val} TAB"
                                                        df_veritrade.loc[idx, "PRESENTACION"] = cadena
                                                        salir = True
                                                        break
                                                    elif (df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="COMPRIMIDOS" or df_veritrade.loc[idx, "FORMA_FARMACEUTICA"]=="COMPRIMIDOS MASTICABLES"):
                                                        cadena = f"{sena} X {val} COMP"
                                                        df_veritrade.loc[idx, "PRESENTACION"] = cadena
                                                        salir = True
                                                        break
                                                else:
                                                    cadena = f"{sena} X {val} {parte_um}"
                                                    df_veritrade.loc[idx, "PRESENTACION"] = cadena
                                                    salir = True 
                                                    break
                                        if salir:  # Verifica si se debe salir después del bucle de valor
                                            break
                                if salir:  # Verifica nuevamente después del bucle de sena
                                    break

                                    #STARTS WITH Y CONTAINS

            except (ValueError, SyntaxError):
                print("Error al evaluar la cadena.")

In [48]:
from pyspark import StorageLevel

df_veritrade_spark_ite8.persist(StorageLevel.MEMORY_AND_DISK)

df=df_veritrade_spark_ite8.toPandas()

df.to_excel("output_geriax.xlsx",index=False)

24/11/19 13:06:37 WARN DAGScheduler: Broadcasting large task binary with size 1153.0 KiB
24/11/19 13:06:46 WARN DAGScheduler: Broadcasting large task binary with size 1156.7 KiB


In [49]:
df.head(1)

,Partida Aduanera,Descripcion de la Partida Aduanera,Aduana,DUA / DAM,Fecha,Cod. Tributario,Exportador,Importador,Kg Bruto,Kg Neto,...,VAR8,DETALLE_VAR8,NUMERO_DE_PALABRAS,DES_PROD,DES_PROD2,DES_PROD3,DES_PROD4,DES_PROD5,PRESENTACION,UNIDAD
0,3004502000,DEMAS MEDICAMENTOS P` USO VETERINARIO Q` CONTE...,MARITIMA DEL CALLAO,005393 | 3,2021-01-25,20250406941,AGROVET MARKET S.A,QUALITY CO. FOR VET. MED. TRADE,12.15,11.54,...,PREVENCION_ENFERMEDAD,NaN,3.0,ARTROSAMINE TABLETAS PALATABLES,ARTROSAMINE TABLETAS PALATABLES,ARTROSAMINE TABLETAS PALATABLES,ARTROSAMINETABLETASPALATABLES,ARTROSAMINETABLETASPALATABLES,None,[]


from pyspark import StorageLevel

df_veritrade_spark_ite6.persist(StorageLevel.MEMORY_AND_DISK)

df_veritrade_spark_ite6.write.csv("output.csv", mode='overwrite')

#PARA LOS QUE NO ENCUENTRA EN SENASA
indices_no_encontrado = df_veritrade[df_veritrade["FK_SENASA_ID"] == ""].index.tolist()

#SE AMARRA UN NO ENCONTRADO CON ALIMENTO
def verificar_empieza_alimento(texto):
    if texto.startswith("ALIMENTO"):
        return "Alimento"
    elif texto.startswith("ALMNTO."):
        return "Alimento"
    else:
        return ""
    
def verificar_contiene_alimento(texto):
    return "ALIMENTO" in texto

for idx in indices_no_encontrado:
    desc_comer_raw = df_veritrade.loc[idx, "Descripcion Comercial"].strip().upper()
    if verificar_empieza_alimento(desc_comer_raw):
        df_veritrade.loc[idx, "TIPO_PRODUCTO"] = "Alimento"
        continue

    if verificar_contiene_alimento(desc_comer_raw):
        df_veritrade.loc[idx, "TIPO_PRODUCTO"] = "Alimento"

#PARA LOS QUE NO ENCUENTRA EN SENASA
indices_no_encontrado = df_veritrade[df_veritrade["FK_SENASA_ID"] == ""].index.tolist()
    
def verificar_contiene_agricola(texto):
    if "AGRICOLA" in texto:
        return True
    elif "AGRÍCOLA" in texto:
        return True
    elif "AGRICULTURA" in texto:
        return True
    else:
        return False

for idx in indices_no_encontrado:
    desc_comer_raw = df_veritrade.loc[idx, "Descripcion Comercial"].strip().upper()  # Normalizar la descripción
    filtro_avanzado=df_veritrade.loc[idx, "TIPO_PRODUCTO"]
    if verificar_contiene_agricola(desc_comer_raw) and filtro_avanzado=="":
        df_veritrade.loc[idx, "TIPO_PRODUCTO"] = "Agrícola"

#df_veritrade.to_excel("VERITRADE_pruebas_01102024.xlsx")

df_veritrade_spark_ite4=df_veritrade_spark_ite3
df_veritrade_spark_ite4=df_veritrade_spark_ite4.coalesce(4)
df_veritrade_spark_ite4.cache()

In [50]:
#num_particiones_veritrade = df_veritrade_spark_ite3.rdd.getNumPartitions()
#print(f"Número de particiones en df_veritrade_spark: {num_particiones_veritrade}")

In [51]:
#df_veritrade_spark_ite3.explain()

In [52]:
#df_output = df_veritrade_spark_ite6.toPandas()

#df_output.to_excel("output_final.xlsx", index=False)

In [53]:
#df_veritrade_spark_ite6=df_veritrade_spark_ite6.repartition(100)

In [54]:
#df_veritrade_spark_ite6.write.csv("hdfs://localhost:9000/alexander/output.csv", header=True)

In [55]:
#print(df_veritrade_spark_ite6.count())
#print(df_veritrade_spark_ite6.printSchema())

In [56]:
#df_veritrade_spark_ite6.write.csv("output.csv", header=True, mode="overwrite")
#df_veritrade_spark_ite6.coalesce(1).write.csv("hdfs://localhost:9000/alexander/output.csv", header=True)

In [57]:
spark.stop()